# Conjetura de Brocard

### Introducción:
La conjetura de brocard es un problema de teoría de números el cual dice que dados 2 números primos consecutivos P<sub>n</sub> y P<sub>n+1</sub>, si se elevan al cuadrado siendo n>=2 la conjetura nos dice que va a haber por lo menos 4 números primos en ese intervalo.
Se cree que la conjetura es cierta mas sin embargo no se ha hecho alguna demostración.

### Objetivos
En este proyecto se tiene como objetivo crear un programa que pruebe sucesivamente los casos de la conjetura de Brocard, no con el propósito de demostrar su veracidad sino de poner a prueba a ver hasta que ínice 'n' se cumple esta conjetura. Adicionalmente a la herramienta, se explicará la lógica que se usó para construir el programa, desde una perspectiva matemática y no en cuanto a estética u opciones que no infieren en el funcionamiento principal del programa.

---

### 1. Preparación del entorno del script:
Para que el programa funcione correctamente se deben importar algunas librerías tanto propias de python como externas, así como declarar variables globales y algunas funciones para que el usuario interactúe a nivel de consola. Cabe aclarar que este programa fue diseñado específicamente para ser ejecutado por consola y recibir argumentos por ese medio, por lo que si bien se tratará de adaptar algunas cosas para mostrar a efectos prácticos el algoritmo en jupyter, pero sin dejar de lado el enfoque de que el programa es diseñado para consola.

En la primera línea usamos un [shebang](https://en.wikipedia.org/wiki/Shebang_(Unix)), básicamente facilita su uso en sistemas Unix. Posteriormente importamos algunas librerías, siendo [pwntools](https://docs.pwntools.com/en/stable/) y [colorama](https://pypi.org/project/colorama/) librerías que no están dentro de las librerías por defecto de python, se tendrían que instalar con **'pip install $nombre_de_la_librería'**. Podríamos separar las librerías en 2 grupos, las que son meramente para cuestiones estéticas (colorama y os) y las que ayudan a el funcionamiento del programa en cuanto a interacción con el usuario (sys, signal y time), en un punto medio se encuentra la librería 'pwntools'.

Con la función def_handler y las librerías 'sys' y 'signal' definimos una salida más 'limpia' del programa al momento de detenerlo usando Ctrl+c.

Definimos algunas variables globales, la más importante es la lista de los primos que inicializamos con los 3 primeros primos (2, 3 y 5) esto por practicidad para el programa ya que como dice la conjetura de Brocard, aplica para primos con ínice mayor o igual a 2. La otras variables globales son para representar colores con facilidad posteriormente, para esto usamos la librería 'colorama'.

Por último definimos una función para limpiar la pantalla al momento de ejecutar el script, está implementado para funcionar tanto en sistemas Windows como en sistemas Unix, para esto utilizamos la librería 'os'.

In [2]:
#!/usr/bin/env python3

"""
Hecho por: Alvaro Camilo Torres
Institución: Universidad Nacional de Colombia
Materia: Matemáticas Discretas
"""
from pwn import *
from colorama import Fore, Style
import sys, signal, time, os

def def_handler(sig, frame):
    print(f"\n\n[{rojo}!{reset}] {rojo}Saliendo...{reset}\n")
    sys.exit(1)

# Ctrl + C
signal.signal(signal.SIGINT, def_handler)

# Variables globales
primos = [2, 3, 5] # La lista de primos se inicializa con los 3 primeros primos para que funcione el programa

#Colores
amarillo = Fore.YELLOW
azul = Fore.BLUE
rojo = Fore.RED
verde = Fore.GREEN
reset = Style.RESET_ALL # Resetea el color de la consola

# Detecta el sistema operativo para limpiar la pantalla
def limpiar_pantalla():
    if os.name == 'nt':
        os.system('cls')
    else:
        os.system('clear')

### 2. Hallar primos:
Es una de las funciones 'nucleo' del programa, la función recibe 2 argumentos, el número a probar si es primo o no, y la lista de números primos que declaramos arriba. Su funcionamiento consiste en suponer que el número es primo e ir iterando por la lista de primos que tenemos definida, si el módulo del número con algún primo de la lista de primos da cero, se dice que el número no es primo (por definición que solo es divisible por 1 y él mismo, y que un número se puede expresar en términos de números primos) y se devuelve el valor que niega que sea primo en una variable 'es_primo' que definimos más arriba como verdadera. La función deja de iterar cuando el primo a probar supera el valor de la raiz cuadrada del número a probar, esto debido a que ese va a ser el máximo divisor que pueda tener ese número y probar con los siguientes sería innecesario.

Se puede intuir al leer el código que esta función es inútil si la lista de primos es muy limitada en comparación al número al que queremos hacerle la prueba, y la razón de esto es que esta función solo será útil en conjunto de otras funciones que la tienen como argumento.

In [3]:
# Indica si un número es primo o no
def hallar_primos(número, primos):
    es_primo = True
    for j in primos:
        if j > int(número**0.5) + 1:
            break
        elif número % j == 0:
            es_primo = False
            break
    return es_primo

### 3. Evaluar conjetura de Brocard:
Otra de las funciones 'nucleo', y esta en particular es la que va a definir toda la lógica del programa. Recibe 3 argumentos, la lista de primos, el índice 'n' a evaluar y la función 'hallar_primos'.

En esta función solucionamos el problema que teníamos de la función 'hallar_primos' que la lista se quedaba corta. Como primera funcionalidad que tiene el programa es que si el ínice 'n' a probar es más grande que la longitud de la lista de primos menos 2 (para que funcione el programa necesitamos el n siguiente), entonces hasta que deje de ser más grande se hallan primos con la función hallar primos, de 1 en 1 desde el primo número 3 que es el último que tenemos definido en nuestra lista global.

Manejamos de manera global 3 variables, el valor de P<sub>n</sub><sup>2</sup>, P<sub>n+1</sub><sup>2</sup> y los primos que están entre esos valores, esto para futura representación.

Ya que tenemos que hallar los primos intermedios, en nuestrá lista de primos de tenemos que tener como mínimo hasta que los primos sean más grandes que el límite superior del intervalo que queremos evaluar, para esto empleamos una lógica similar a la que usamos cuando el índice 'n' era más grande que los primos que teníamos en la lista, solo que ahora prueba números 1 en 1 desde el último primo hasta el límite superior con la función 'hallar_primos'. Por último se ingresan los primos intermedios en una lista iterando por todo el intervalo que vamos a evaluar y revisando si el número está en nuestra lista de primos.

In [5]:
# Encuentra los primos entre P_n^2 y P_(n+1)^2
def brocard(primos, n, hallar_primos):
    
    global tope_superior, tope_inferior, intermedios
    i = 1

    while True:
        if n > len(primos) - 2:
            if hallar_primos(primos[2] + i, primos):
                primos.append(primos[2] + i)
            i += 1
        else:
            break

    tope_superior = primos[n + 1] ** 2
    tope_inferior = primos[n] ** 2
    intermedios = []
   

    if tope_superior > primos[-1]:
        for num in range(primos[-1] + 1, tope_superior + 1):
            if hallar_primos(num, primos):
                primos.append(num)

    intermedios = [str(i) for i in range(tope_inferior, tope_superior + 1) if i in primos]

### 4. Funcionalidades del programa:
Ya con las funciones que definen nuestra lógica del lado del usuario vamos a usar esta lógica para representar la conjetura de Brocard. La primera funcionalidad que definimos es la más lógica para representar el problema, una función que va representando la conjetura incrementando progresivamente el índice 'n' infinitamente. Esto simplemente consiste en hacer la función brocard en bucle infinito incrementando 'n' de 1 en 1. La demás parte del código simplemente infiere en la representación gráfica por consola

La segunda funcionalidad está enfocada a representar la conjetura de Brocard indicando por consola el índice 'n' para el cual queremos probar.

In [6]:
# Evalúa la conjetura de Brocard incrementando el ínice n infinitamente
def brocard_infinito(primos, hallar_primos, brocard, velocidad):
    proc1 = log.progress("")
    proc1.status("Iniciando prueba a la conjetura de Brocard")
       
    if '-v' in sys.argv or '--verbose' in sys.argv:
        proc2 = log.progress("")
        proc2.status("Listando primos entre P_n^2 y P_(n+1)^2")
    n = 1

    time.sleep(2)
   
    while True:
        brocard(primos, n, hallar_primos)
        if len(intermedios) < 4:
            log.info(f" Se ha encontrado una excepción al problema de brocard con el primo {azul}{primos[n]}{reset} donde entre {azul}{primos[n]**2}{reset} y {azul}{primos[n+1]**2}{reset} no hay 4 primos")
            break
        else:
            proc1.status(f"Para {amarillo}P_{n}{reset} = {azul}{primos[n]}{reset}, entre {azul}{tope_inferior}{reset} y {azul}{tope_superior}{reset} se encontraron {verde}{len(intermedios)}{reset} primos")
            n += 1
        if '-v' in sys.argv or '--verbose' in sys.argv:
            proc2.status(f"Los primos encontrados son: {azul}{(reset +', ' + azul).join(intermedios)}{reset}")

        if velocidad == 1:
            time.sleep(2)
        elif velocidad == 2:
            time.sleep(1)
        elif velocidad == 3:
            time.sleep(0.5)        

# Evalúa la conjetura de Brocard para un ínice 'n' en particular
def brocard_individual(primos, hallar_primos, brocard, n):
   
    brocard(primos, n, hallar_primos)
    
    log.info(f"Para {amarillo}P_{n}{reset} = {azul}{primos[int(n)]}{reset}, entre {azul}{tope_inferior}{reset} y {azul}{tope_superior}{reset} se encontraron {verde}{len(intermedios)}{reset} primos")
    if '-v' in sys.argv or '--verbose' in sys.argv:
        log.info(f"Los primos encontrados son: {azul}{(reset +', ' + azul).join(intermedios)}")

### 5. Uso del programa:
El enfoque del programa es de uso por consola, por lo que está configurado para recibir argumentos por consola, estos ejecutan las diferentes funcionalidades implementadas en el programa y las pueden configurar dándoles diferentes argumentos. Por esta razón el programa puede no correr en este notebook y lo ideal sería importar las librerías y correrlo en local.
En cuanto al estilo del programa y su uso, me basé en scripts de python que funcionan como herramientas de ciberseguridad (personalmente me gusta mucho esa área de la informática), donde casi por convención se suelen usar argumentos por consola para ejecutarlos, así como otros aspectos de estilo.

In [7]:
# Muestra las instrucciones del programa por consola
def panel_ayuda():
    print(f"""\n\n[{verde}+{reset}] Modo de uso:

        python3 brocard.py [{amarillo}opciones{reset}] [{amarillo}argumentos{reset}]

[{verde}+{reset}] Ejemplos:

        python3 brocard.py -h
        python3 brocard.py -i 3
        python3 brocard.py -n 100

[{verde}+{reset}] Opciones:

        {amarillo}-h{reset}, {amarillo}--help{reset}                  Muestra este panel de ayuda

        {amarillo}-i{reset}, {amarillo}--infinite{reset} {azul}speed{reset}        Prueba la conjetura de Brocard infinitamente hasta que pare el programa.
                                    Como argumento puede recibir un entero del 1-4 para representar velocidad, siendo 1 la más lenta y 4 la más rápida(por defecto)

        {amarillo}-n{reset}, {amarillo}--index{reset} {azul}int{reset}             Prueba el programa con un índice n en específico    
        {amarillo}-v{reset}, {amarillo}--verbose{reset}               Muestra los primos entre el intervalo P_n^2 y P_(n+1)^2
        """)

### 6. Ejecución:
Al ejecutarse el script por defecto lanzará el panel de ayuda para ver las distintas opciones de uso y dependiento de los argumentos que se le suministren por consola el programá se comportará de forma diferente. El condicional que hace referencia al main es en caso de que alguien quiera usar las funciones definidas en el programa pueda importar el script como módulo sin que se ejecute el programa principal, básicamente solo funciona si se ejecuta directamente y no como módulo, esto para darle un enfoque modular al código. Por último para efectos visuales en el notebook añadiré la ejecución de brocard_individual(importar pwntools y colorama para ver funcionamiento).

In [8]:
if __name__ == '__main__': # Si se ejecuta el script directamente
    
    limpiar_pantalla()
    print(f"""{verde} ______                                               __  _                  
|_   _ \                                             |  ]| |                 
  | |_) | _ .--.   .--.   .---.  ,--.   _ .--.   .--.| | \_|.--.             
  |  __'.[ `/'`\]/ .'`\ \/ /'`\]`'_\ : [ `/'`\]/ /' \  |   ( (`\]            
 _| |__) || |    | \__. || \__. // | |, | |    | \__/  |    `'.'.            
|_______/[___]    '.__.' '.___.'\ -;__/[___]    '.__.;__]  [\__) )           
                                _                _                           
                               (_)              / |_                         
    .---.   .--.   _ .--.      __  .---.  .---.`| |-'__   _   _ .--.  .---.  
   / /'`\]/ .'`\ \[ `.-. |    [  |/ /__ \/ /'`\]| | [  | | | [ `/'`\]/ /__ \ 
   | \__. | \__. | | | | |  _  | || \__.,| \__. | |, | \_/ |, | |    | \__., 
   '.___.' '.__.' [___||__][ \_| | '.__.''.___.'\__/ '.__.'_/[___]    '.__.' 
                            \____/                                           {reset}\n""")

    if len(sys.argv) == 1 or '-h' in sys.argv or '--help' in sys.argv:

        panel_ayuda()

    elif sys.argv[1] == '-i' or sys.argv[1] == '--infinite':
        try:
            brocard_infinito(primos, hallar_primos, brocard, 4 if len(sys.argv) == 2 or (('-v' in sys.argv or '--verbose' in sys.argv) and len(sys.argv) == 3) else int(sys.argv[2]))
        except ValueError:
            panel_ayuda()
    
    elif sys.argv[1] == '-n' or sys.argv[1] == '--index':
        try:
            brocard_individual(primos, hallar_primos, brocard, int(sys.argv[2]))
        except ValueError:
            panel_ayuda()

    # Exclusivo para este notebook:
    brocard_individual(primos, hallar_primos, brocard, 5) # Ejemplo para n=5
    time.sleep(3)
    brocard_infinito(primos, hallar_primos, brocard, 1)

 ______                                               __  _                  
|_   _ \                                             |  ]| |                 
  | |_) | _ .--.   .--.   .---.  ,--.   _ .--.   .--.| | \_|.--.             
  |  __'.[ `/'`\]/ .'`\ \/ /'`\]`'_\ : [ `/'`\]/ /' \  |   ( (`\]            
 _| |__) || |    | \__. || \__. // | |, | |    | \__/  |    `'.'.            
|_______/[___]    '.__.' '.___.'\ -;__/[___]    '.__.;__]  [\__) )           
                                _                _                           
                               (_)              / |_                         
    .---.   .--.   _ .--.      __  .---.  .---.`| |-'__   _   _ .--.  .---.  
   / /'`\]/ .'`\ \[ `.-. |    [  |/ /__ \/ /'`\]| | [  | | | [ `/'`\]/ /__ \ 
   | \__. | \__. | | | | |  _  | || \__.,| \__. | |, | \_/ |, | |    | \__., 
   '.___.' '.__.' [___||__][ \_| | '.__.''.___.'\__/ '.__.'_/[___]    '.__.' 
                            \____/                              

KeyboardInterrupt: 

### Código a futuro:
Dado a que se diseño el código con una estructura modular, está abierto a mejoras posteriores al código. El código está público en mi repositorio de Github [aquí](https://github.com/4lv4r0t/Conjetura-de-Brocard), es free software por lo que cualquiera es libre de realizar sus propias modificaciones al mismo.

### Información de verisón y hardware:
Python implementation: CPython

Python version       : 3.12.2

IPython version      : 8.21.0


OS: Arch Linux x86_64 

Host: VivoBook_ASUSLaptop X421IAY_M413IA 1.0 

Kernel: 6.6.7-arch1-1 

Shell: zsh 5.9 

Terminal: kitty 

CPU: AMD Ryzen 5 4500U with Radeon Graphics (6) @ 2.375GHz 

GPU: AMD ATI Radeon RX Vega 6 

Memory: 4343MiB / 7357MiB 

---
**Autor:** Alvaro Camilo Torres Rodríguez

**Institución:** Universidad Nacional de Colombia